In [1]:
import numpy as np
import pandas as pd

In [2]:
def prepare_problem(xs, ys, quantile):
    X_train, y_train = xs.to_frame(), ys
    X = np.concatenate([
        np.full(X_train.shape[0], 1.0).reshape(-1, 1),
        X_train.to_numpy()
    ], axis=1)
    n, k = X.shape

    c = np.concatenate([
        np.full(2 * k, 0.0),
        quantile * np.full(n, 1.0),
        (1 - quantile) * np.full(n, 1.0)
    ])
    A = np.concatenate([
        X, -X, np.identity(n), -np.identity(n)
    ], axis=1)
    m = A.shape[1]
    b = y_train.to_numpy().reshape(-1, 1)

    return c, A, b, m, n, k

In [3]:
def steepest_edge_simplex(_c, _A, _b, steepest_edge=True, iterations_limit=1_000):
    # See: D. Goldfarb, J. K. Reid, "A practicable steepest-edge simplex algorithm"
    '''
    Problem:
    T(c) * x -> min
    Constraints:
    A * x = b
    x >= 0
    '''
    from numpy import linalg as la

    def get_init_matrices():
        c = np.array(_c).astype(float)
        if len(c.shape) == 1:
            c = c.reshape(-1, 1)
        A = np.array(_A).astype(float)
        b = np.array(_b).astype(float)
        if len(b.shape) == 1:
            b = b.reshape(-1, 1)
        return c, A, b

    def get_pivot_indices(A):
        from sympy import Matrix

        return Matrix(A).rref()[1]

    def split_A(init_A, pivot_indices, dependent_indices):
        A1 = init_A[:, pivot_indices]
        A2 = init_A[:, dependent_indices]
        A = np.block([[A1, A2]])
        return A1, A2, A

    def split_c(init_c, pivot_indices, dependent_indices):
        c1 = init_c[pivot_indices, :]
        c2 = init_c[dependent_indices, :]
        c = np.block([[c1], [c2]])
        return c1, c2, c

    def get_row(A, i):
        return np.array([A[i, :]])

    def get_column(A, i):
        return A[:, i].reshape(-1, 1)

    def e(n, i):
        return np.eye(n)[:, i].reshape(-1, 1)

    def permutation_matrix(n, i, j):
        P = np.eye(n)
        P[:, [i, j]] = P[:, [j, i]]
        return P

    c, A, b = get_init_matrices()
    m, n = A.shape

    def construct_N(A1, A2, A1_inv):
        N = np.block([[A1, A2], [np.zeros((n - m, m)), np.eye(n - m)]])
        N_inv = np.block([[A1_inv, -np.dot(A1_inv, A2)], [np.zeros((n - m, m)), np.eye(n - m)]])
        return N, N_inv

    def is_optimal(z):
        return np.all(z[m:, :] >= 0)

    def get_pivot_column_index(z, gamma):
        max_value = float('-inf')
        q = -1
        for i in range(m, n):
            z_i = z[i, 0]
            if z_i >= 0:
                continue
            value = z_i**2 / gamma[i, 0]
            if value > max_value:
                max_value = value
                q = i
        return q

    def is_unbounded(w):
        return np.all(w <= 0)

    def get_pivot_index(x, w):
        min_value = float('inf')
        p = -1
        for i in range(m):
            w_i = w[i, 0]
            if w_i <= 0:
                continue
            value = x[i, 0] / w_i
            if value < min_value:
                min_value = value
                p = i
        return p

    def get_alpha(A, N_inv, revised_A1_inv, p, q):
        alpha = -get_row(N_inv, p).reshape(-1, 1)
        y = np.dot(revised_A1_inv, e(m, p))
        revised_alpha = alpha.copy()
        for i in range(m, n):
            if i == q:
                continue
            revised_alpha[i, 0] = np.dot(y.T, get_column(A, i))[0, 0]
        return revised_alpha

    def revise_x(x, w, w_p, p):
        x_p = x[p, 0] / w_p
        revised_x = x - np.block([[w * x_p], [np.zeros((n - m, 1))]])
        revised_x[p, 0] = x_p
        return revised_x

    def revise_z(z, alpha, w_p, z_q, p, q):
        revised_z = z.copy()
        for i in range(n):
            if i < m:
                continue
            if i == q:
                revised_z[i, 0] = -z_q / w_p
            else:
                revised_z[i, 0] = z[i, 0] - alpha[i, 0] * z_q
        return revised_z

    def revise_gamma(gamma, alpha, wTA, w_p, gamma_q, p, q):
        revised_gamma = gamma.copy()
        for i in range(n):
            if i < m:
                continue
            if i == q:
                revised_gamma[i, 0] = gamma_q / w_p**2
            else:
                revised_gamma[i, 0] = max(
                    gamma[i, 0] - 2 * alpha[i, 0] * wTA[i, 0] + alpha[i, 0]**2 * gamma_q,
                    1 + alpha[i, 0]**2
                )
        return revised_gamma

    assert m < n, "Bad shape of matrix A: {0}".format(A.shape)
    assert b.shape[0] == m and b.shape[1] == 1, "Bad shape of matrix b: {0}".format(b.shape)
    assert c.shape[0] == n and c.shape[1] == 1, "Bad shape of matrix c: {0}".format(c.shape)

    pivot_indices = get_pivot_indices(A)
    assert len(pivot_indices) == m, \
        "Constraint matrix should have {0} independent columns, but have only {1}".format(m, len(pivot_indices))
    dependent_indices = tuple(set(range(n)) - set(pivot_indices))
    c1, c2, c = split_c(c, pivot_indices, dependent_indices)
    A1, A2, A = split_A(A, pivot_indices, dependent_indices)
    A1_inv = la.inv(A1)
    x = np.block([[np.dot(A1_inv, b)], [np.zeros((n - m, 1))]])
    N, N_inv = construct_N(A1, A2, A1_inv)
    z = np.dot(c.T, N_inv).T
    if steepest_edge:
        gamma = np.diag(np.dot(N_inv.T, N_inv)).reshape(-1, 1)
    else:
        gamma = np.full((n, 1), 1.0)

    iteration_number = 0

    while True:
        # (a)
        if is_optimal(z):
            print("Stopped at {0}".format(iteration_number))
            indices = np.array(pivot_indices + dependent_indices).reshape(-1, 1)
            x_indexed = np.block([x, indices])
            x_sorted = x_indexed[x_indexed[:, 1].argsort()]
            return x_sorted[:, 0]
        # (b)
        q = get_pivot_column_index(z, gamma)
        w = np.dot(A1_inv, get_column(A, q))
        # (c)
        if is_unbounded(w):
            print("Stopped at {0}".format(iteration_number))
            print("Problem is unbounded")
            return None
        # (d)
        p = get_pivot_index(x, w)
        w_p = w[p, 0]
        # (e)
        revised_x = revise_x(x, w, w_p, p)
        # (f)
        z_q = c[q, 0] - np.dot(c1.T, w)[0, 0]
        if steepest_edge:
            gamma_q = 1.0 + np.dot(w.T, w)[0, 0]
            w = np.dot(A1_inv.T, w)
        # revise indices
        pivot_new_index_at_p = dependent_indices[q - m]
        dependent_new_index_at_q = pivot_indices[p]
        pivot_indices = tuple([(pivot_new_index_at_p if i == p else idx) for i, idx in enumerate(pivot_indices)])
        dependent_indices = tuple([(dependent_new_index_at_q if i == q - m else idx) for i, idx in enumerate(dependent_indices)])
        # revise c, A, N
        init_c, init_A, init_b = get_init_matrices()
        revised_c1, revised_c2, revised_c = split_c(init_c, pivot_indices, dependent_indices)
        revised_A1, revised_A2, revised_A = split_A(init_A, pivot_indices, dependent_indices)
        revised_A1_inv = la.inv(revised_A1)
        revised_N, revised_N_inv = construct_N(revised_A1, revised_A2, revised_A1_inv)
        # revised alpha
        revised_alpha = get_alpha(A, N_inv, revised_A1_inv, p, q)
        # revise z and gamma
        revised_z = revise_z(z, revised_alpha, w_p, z_q, p, q)
        if steepest_edge:
            revised_gamma = revise_gamma(gamma, revised_alpha, np.dot(w.T, A).reshape(-1, 1), w_p, gamma_q, p, q)

        # revision
        x = revised_x.copy()
        c1, c2, c = revised_c1.copy(), revised_c2.copy(), revised_c.copy()
        A1, A2, A = revised_A1.copy(), revised_A2.copy(), revised_A.copy()
        A1_inv = revised_A1_inv.copy()
        N, N_inv = revised_N.copy(), revised_N_inv.copy()
        z = revised_z.copy()
        if steepest_edge:
            gamma = revised_gamma.copy()

        iteration_number += 1
        if iteration_number == iterations_limit:
            print("Iterations limit achieved")
            return None

c = [1, 0, 0, 0, -1]
A = pd.DataFrame({'x1': [0, 0, 0], 'x2': [0, 0, 1], 'x3': [0, 1, 0], 'x4': [1, 0, 0], 'x5': [0, 0, 1]})
b = np.array([[0], [1], [2]])
steepest_edge_simplex(c, A, b)

Stopped at 1


array([0., 0., 1., 0., 2.])

In [4]:
class InteriorPointSolver:
    from numpy.linalg import matrix_rank
    import sympy

    def solve(self, c, A, b, epsilon=0.0001):
        # ensure dimensions are okay
        assert A.shape[0] == b.shape[0], 'first dims of A and b must match, check input!'
        assert A.shape[1] == c.shape[0], 'second dim of A must match first dim of c, check input!'

        # ensure A is full rank, drop redundant rows if not
        if self.matrix_rank(A) < min(A.shape[0], A.shape[1]):
            _, pivots = self.sympy.Matrix(A).T.rref()
            A = A[list(pivots)]

        m = A.shape[0]
        n = A.shape[1]

        # initial solution (x_0, lambda_0, s_0) > 0 [lambda is variable l in code]
        x = np.ones(shape=(n, ))
        l = np.ones(shape=(m, ))
        s = np.ones(shape=(n, ))

        # set iteration counter to 0 and mu_0
        k = 0

        # main loop body
        while abs(np.dot(x, s)) > epsilon:
            # increase iteration number
            k += 1

            # choose sigma_k and calculate mu_k
            sigma_k = 0.4
            mu_k = np.dot(x, s) / n

            # create linear system A_ * delta = b_
            A_ = np.zeros(shape=(m + n + n, n + m + n))
            A_[0:m, 0:n] = np.copy(A)
            A_[m:m + n, n:n + m] = np.copy(A.T)
            A_[m:m + n, n + m:n + m + n] = np.eye(n)
            A_[m + n:m + n + n, 0:n] = np.copy(np.diag(s))
            A_[m + n:m + n + n, n + m:n + m + n] = np.copy(np.diag(x))

            b_ = np.zeros(shape=(n + m + n, ))
            b_[0:m] = np.copy(b - np.dot(A, x))
            b_[m:m + n] = np.copy(c - np.dot(A.T, l) - s)
            b_[m + n:m + n + n] = np.copy( sigma_k * mu_k * np.ones(shape=(n, )) - np.dot(np.dot(np.diag(x), np.diag(s)), np.ones(shape=(n, ))) )

            # solve for delta
            delta = np.linalg.solve(A_, b_)
            delta_x = delta[0:n]
            delta_l = delta[n:n + m]
            delta_s = delta[n + m:n + m + n]

            # find step-length alpha_k
            alpha_max = 1.0
            for i in range(n):
                if delta_x[i] < 0:
                    alpha_max = min(alpha_max, -x[i]/delta_x[i])
                if delta_s[i] < 0:
                    alpha_max = min(alpha_max, -s[i]/delta_s[i])
            eta_k = 0.99
            alpha_k = min(1.0, eta_k * alpha_max)

            # create new iterate
            x = x + alpha_k * delta_x
            l = l + alpha_k * delta_l
            s = s + alpha_k * delta_s

        return x

In [5]:
def compare(xs, ys, quantile, r=3):
    c, A, b, m, n, k = prepare_problem(xs, ys, quantile)
    beta1 = steepest_edge_simplex(c, A, b)
    if beta1 is not None:
        print((beta1[0:k] - beta1[k:2*k]).round(r))
    beta2 = InteriorPointSolver().solve(c, A, b.reshape(-1))
    print((beta2[0:k] - beta2[k:2*k]).round(r))
    return beta1, beta2

In [6]:
df = pd.DataFrame({
    "x": [0, 0, 1, 1, 1, 2, 2, 3],
    "y": [0, 1, 0, 1, 2, 1, 2, 3],
})
df.head()

,x,y
0,0,0
1,0,1
2,1,0
3,1,1
4,1,2


In [7]:
_ = compare(df.x, df.y, .01)

Stopped at 1
Problem is unbounded
[-1.  1.]


In [8]:
_ = compare(df.x, df.y, .25)

Stopped at 1
Problem is unbounded
[-0.   0.5]


In [9]:
_ = compare(df.x, df.y, .5)

Stopped at 1
Problem is unbounded
[0. 1.]


In [10]:
_ = compare(df.x, df.y, .75)

Stopped at 1
Problem is unbounded
[1.    0.667]


In [11]:
_ = compare(df.x, df.y, .99)

Stopped at 1
Problem is unbounded
[1.5 0.5]
